# Setting default visualisations for simulations in NanoVer's OmniRunner

In [9]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation
from nanover.app import RenderingSelection
from nanover.app.client import SELECTION_ROOT_ID
from nanover.mdanalysis import frame_data_to_mdanalysis

In [7]:
def get_selection_indices(frame, query):
    # use frame's topology to construct an mdanalysis universe
    universe = frame_data_to_mdanalysis(frame)
    # query mdanalysis universe for desired atoms
    atoms = universe.select_atoms(query)
    # convert to integer atom indices
    indices = map(int, atoms.indices)
    return indices

In [8]:
def make_solvent_hiding_selection(omni_omm_simulation):
    omni_omm_simulation.load()
    frame = omni_omm_simulation.dump_frame()
    solvent_indices = get_selection_indices(frame, "resname HOH")
    
    selection = RenderingSelection("solvent", "Solvent")
    selection.set_particles(solvent_indices)
    selection.hide = True
    selection.interaction_method = "none"

In [10]:
nanotube_xml = "openmm_files/nanotube.xml"
alanine_xml = "openmm_files/17-ala.xml"

omni_nanotube = OpenMMSimulation.from_xml_path(nanotube_xml)
omni_alanine = OpenMMSimulation.from_xml_path(alanine_xml)

nanotube_selection = RenderingSelection(SELECTION_ROOT_ID)
nanotube_selection.renderer = {
    'render': 'ball and stick',
    'color': {'type': 'particle index', 'gradient': ["Orange", "Blue"]}
}

alanine_selection = RenderingSelection(SELECTION_ROOT_ID)
alanine_selection.renderer = {
    'color': 'Gold',
    'scale': 0.1,
    'render': 'liquorice'
}

In [11]:
# This will just automatically close previous instances of the runner if you re-run this cell multiple times
try:
    omni_runner.close()
except NameError:
    pass
omni_runner = OmniRunner.with_basic_server(port=0)

omni_runner.add_simulation(omni_nanotube)
omni_runner.set_simulation_selections(omni_nanotube, nanotube_selection, make_solvent_hiding_selection(nanotube_selection))

omni_runner.add_simulation(omni_alanine)
omni_runner.set_simulation_selections(omni_alanine, alanine_selection)

omni_runner.next()

print(f'{omni_runner.app_server.name}: serving at {omni_runner.app_server.address}:{omni_runner.app_server.port}')

AttributeError: 'RenderingSelection' object has no attribute 'load'

# Close the Server

In [4]:
omni_runner.close()